[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gdslab/d2spy/blob/main/docs/guides/notebooks/07_3dep_search_by_d2s_projects.ipynb)
[![Jupyter Notebook](https://img.shields.io/badge/Open%20in%20JuypterHub%20-%20%233776AB?logo=jupyter&logoColor=%23F37626&labelColor=%23F5F5F5)](https://lab.d2s.org/hub/user-redirect/lab/tree/tutorials/07_3dep_search_by_d2s_projects.ipynb)

# 3DEP search by D2S projects
*This guide will walk you through the steps to search a STAC Catalog using the STAC API to find 3DEP items within a D2S project.*

To get started, you'll need access to a D2S instance with existing projects. In addition to [`d2spy`](https://py.d2s.org), this guide will use Python packages [`pystac_client`](https://pystac-client.readthedocs.io/en/stable/) and [`numpy`](https://numpy.org/install/).

In [ ]:
# Uncomment and run the following line if working out of Google Colab
# !pip install pystac_client
# !pip install numpy
# !pip install d2spy

In [ ]:
import numpy as np
from pystac_client import Client

from d2spy.workspace import Workspace

You must connect to your D2S workspace before you can request any data. The [`Workspace`](https://py.d2s.org/workspace/) module's [`connect`](https://py.d2s.org/workspace/#d2spy.workspace.Workspace.connect) method can be used to login to a D2S instance and connect to your workspace in one go.

*Note: This tutorial uses a D2S instance hosted at https://ps2.d2s.org. You will need to have an account and access to data on this instance to use it. Change the URL if you are self-hosting an instance or using an instance hosted elsewhere.*

In [ ]:
# Connect to D2S workspace
workspace = Workspace.connect("https://ps2.d2s.org", "yourD2Semail@example.com")

Locate the project you're interested in for 3DEP data.

In [ ]:
# Change the search term in `.filter_by_title` to match your project
project = workspace.get_projects().filter_by_title("INDOT")[0]
project_boundary = project.get_project_boundary()

In an upcoming step, you will perform a spatial query using STAC API to find 3DEP items located within your project. This query will require providing STAC API with a bounding box, `[xmin, ymin, xmax, ymax]` for the project.

In [ ]:
# Load project boundary coordinates as numpy array
boundary_arr = np.array(project_boundary["geometry"]["coordinates"][0])
# Create a bounding box for the project
bounding_box = [
    boundary_arr[:, 0].min(),
    boundary_arr[:, 1].min(),
    boundary_arr[:, 0].max(),
    boundary_arr[:, 1].max()
]

D2S provides a STAC API that hosts a 3DEP collection for you to search. The API is accessible at https://stac-api.d2s.org. For a more user-friendly interface to browse the data exposed by the API, visit https://stac.d2s.org. In the following cells, you'll connect to the STAC API using pystac_client and search the 3DEP collection using your bounding box.

In [ ]:
# Connect to STAC API
client = Client.open("https://stac-api.d2s.org")

In [ ]:
# Search 3DEP collection
search = client.search(
    max_items=10,
    collections=["3dep"],
    bbox=bounding_box,
)

In [ ]:
# Print STAC Item ID and STAC Browser URL for search results
stac_browser_base_item_url = "https://stac.d2s.org/collections/3dep/items"
for item in search.items():
    print(f"ID: {item.id}, URL: {stac_browser_base_item_url}/{item.id}")
    # You can also directly access the asset URL from the item
    # item.assets["ept.json"].href

Once finished viewing your data, you can revoke your authorization session by logging out.

In [ ]:
workspace.logout()